In [10]:
REFERENCE = 'Reference'
PROJECT = 'Project'

%load_ext tensorboard

from model.model import *
from process_func import *
import process_func_ref as ref
import os
DATA_DIR = f'datasets/'
map = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Military', 
       'Col': 'Military', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Military', 
       'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
       'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
titles =['Miss', 'Mr', 'Mrs', 'Royal', 'Other', 'Master', 'Military']

# Run for validation

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
prepath = 'preprocessed'
try:
    if not os.path.exists(prepath):
        os.makedirs(prepath)
except OSError:
    print('Error Creating Directory...')
    
data = Titanic(DATA_DIR, show_head = False)
data.Preprocess(map, titles, PROJECT)
data._data.to_csv(prepath + f"/encoded_{PROJECT}.csv", index = False)
dataset = data.GetXandY()

data_ref = ref.Titanic(DATA_DIR, show_head = False)
data_ref.Preprocess(map, titles, REFERENCE)
data_ref._data.to_csv(prepath + f"/encoded_{REFERENCE}.csv", index = False)
dataset_ref = data_ref.GetXandY()

# Run for validation

Initializing...
Data Loaded.
Data Preprocessing...


c:\Users\deep\Documents\GitHub\MLBinaryClassification\process_func.py:97: RuntimeWarning: invalid value encountered in double_scalars
  cnt /= len(group_df.loc[group_df['Type'] != 'test'])


Done Preprocessing.
Returned Data Dictionary
Initializing...
Data Loaded.
Data Preprocessing...
Done Preprocessing.
Returned Data Dictionary


In [12]:
ref_model = ReferenceModel(dataset['x_train'].shape)
CP_dir_ref = SetCheckpoint(REFERENCE)
TB_dir_ref = SetLog(REFERENCE)

Model: "ReferenceModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 19)]              0         
                                                                 
 Dense0 (Dense)              (None, 16)                320       
                                                                 
 Dense1 (Dense)              (None, 8)                 136       
                                                                 
 Output (Dense)              (None, 1)                 9         
                                                                 
Total params: 465
Trainable params: 465
Non-trainable params: 0
_________________________________________________________________


In [13]:
proj_model = ProjectModel(dataset['x_train'].shape)
CP_dir_proj = SetCheckpoint(PROJECT)
TB_dir_proj = SetLog(PROJECT)
# Run for validation

Model: "ProjectModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 19)]              0         
                                                                 
 Dense0 (Dense)              (None, 16)                320       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 batch_normalization_3 (Batc  (None, 16)               64        
 hNormalization)                                                 
                                                                 
 Dense1 (Dense)              (None, 16)                272       
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                      

In [14]:
BATCH_SIZE = 128
EPOCHS = 400
callbacks_ref = DefCallbacks(REFERENCE, CP = CP_dir_ref, TB = TB_dir_ref)
history_ref = ref_model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks_ref, 
          validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 0)

In [15]:
BATCH_SIZE = 128
EPOCHS = 400
callbacks_proj = DefCallbacks(PROJECT, CP = CP_dir_proj, TB = TB_dir_proj)
history_proj = proj_model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks_proj, 
          validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 0)

In [16]:
submission = PerformanceCheck(ref_model, CP_dir_ref, dataset['x_test'], DATA_DIR)
submission.to_csv(f"submission/submission_{REFERENCE}.csv", index = False)
submission = PerformanceCheck(proj_model, CP_dir_proj, dataset['x_test'], DATA_DIR)
submission.to_csv(f"submission/submission_{PROJECT}.csv", index = False)

3/3 [==============================] - 0s 2ms/step
0    0.116092
1    0.570573
2    0.118166
3    0.121795
4    0.844192
5    0.133729
6    0.441133
7    0.118760
8    0.796953
9    0.078735
dtype: float32
ReferenceModel_Accuracy :  79.90 %
3/3 [==============================] - 0s 1ms/step
0    0.130725
1    0.651851
2    0.104890
3    0.137452
4    0.858836
5    0.152318
6    0.537595
7    0.180996
8    0.758120
9    0.037542
dtype: float32
ProjectModel_Accuracy :  79.43 %
